In [1]:
## Score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

## Models
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


In [2]:
import pandas as pd
from data.data import Data

data_raw = Data().get_all_data()

In [3]:
# df1 is fine for PreProcessing


df1 = data_raw['twitter_MBTI']
df1.drop(df1.columns[0], axis=1, inplace=True)
df1 = df1.rename(columns={'text': 'posts', 'label': 'type'})

df2 = data_raw['MBTI 500']

df3 = data_raw['mbti_1']
df3 = df3[['posts', 'type']]




## Combined all data for PP if desired
data_combined = pd.concat([df1,df1,df3], axis=0)
data_combined.head()

,posts,type
0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj
1,@Hispanthicckk Being you makes you look cute||...,intj
2,@Alshymi Les balles sont réelles et sont tirée...,intj
3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj
4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj


While I did spend some time settting this up, the script still
did not work quite yet. I will wait and talk to everyone about this.
Note that you did include a setup file, Makefile, and the appropriate 
__init__.py files in order run the scripts that have been setup.

In [4]:
# from scripts.preprocessing import preprocessing

# data_pp = preprocessing(data_combined)

# TypeError: expected string or bytes-like object

In [5]:
## Data for model selection
data = df2

In [6]:
data['e_i'] = data['type'].astype(str).str[0]
data['s_n'] = data['type'].astype(str).str[1]
data['f_t'] = data['type'].astype(str).str[2]
data['p_j'] = data['type'].astype(str).str[3]

In [7]:
models_list = [SGDClassifier(), RandomForestClassifier(), GradientBoostingClassifier(),
               DecisionTreeClassifier(), KNeighborsClassifier(),SVC(),
               GaussianNB(), MultinomialNB()]

In [8]:
def BaselineModel(X_train, X_test, y_train, model_selection):
    count_vectorizer = CountVectorizer()
    X_bow_train = count_vectorizer.fit_transform(X_train)
    X_bow_test = count_vectorizer.fit_transform(X_test)

    model = model_selection

    model.fit(X_bow_train, y_train)
    
    return model, X_bow_test

In [9]:
## Finds the Macro F1 - Score
def PredictDict(model, X_bow_test, y_test):
    y_pred = cross_val_predict(model, X_bow_test, y_test, cv = 10)
    report = classification_report(y_test, y_pred, output_dict=True)
    return report

In [10]:
y_set = data['e_i']
X_set = data['posts']

X_train, X_test, y_train, y_test = train_test_split(X_set,y_set,
                                                test_size=.3,
                                                random_state=1) 

In [11]:
model, X_bow_test = BaselineModel(X_train, 
                                  X_test, 
                                  y_train, 
                                  MultinomialNB())

In [22]:
## Use our previous code to get F1 Scores for different models
def ModelF1Score_Binary(Column, model):
    """
    Input the Column from the DataFrame as a string for binary modelling.
    Outputs prints of the Confusion Matrix and the Classification Report.
    """
    y_set = data[Column]
    X_set = data['posts']
    
    X_train, X_test, y_train, y_test = train_test_split(X_set,y_set,
                                                    test_size=.3,
                                                    random_state=1) 
    
    model_fit, X_bow_test = BaselineModel(X_train = X_train, 
                                      X_test = X_test, 
                                      y_train = y_train, 
                                      model_selection = model)
    
    print(f"\n{model} has been fit!")
    
    prediction = PredictDict(model = model_fit, 
                       X_bow_test = X_bow_test, 
                       y_test = y_test)
    
    predict_list = list(prediction.keys())
    
    class_1 = predict_list[0]
    class_2 = predict_list[1]
    macro = predict_list[3]
    
    print("----------------------------------------------------------------")
    print(f"{class_1} has a score of: {100*round(prediction[class_1]['f1-score'],4)}%")
    print(f"{class_2} has a score of: {100*round(prediction[class_2]['f1-score'],4)}%")
    print(f"The Combined Macro F1 Average is:{100*round(prediction[macro]['f1-score'],4)}%")
    print("----------------------------------------------------------------")
    